In [1]:
from pathlib import Path
import glob
import numpy as np
from tqdm import tqdm
import open3d as o3d

INFO - 2023-06-28 15:20:25,060 - utils - Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2023-06-28 15:20:25,061 - utils - NumExpr defaulting to 8 threads.


In [50]:
def decimate(DATA_ROOT, TARGET_ROOT, FACES, EXTENSION):
    for file in tqdm(glob.glob(str(DATA_ROOT / "*.ply"))):
        _path = file.replace('\\', '/')

        _id = _path[_path[:_path.rfind(".")].rfind("/")+1:_path.rfind(".",0,)]

        original_mesh = o3d.io.read_triangle_mesh(_path)
        decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(original_mesh, FACES)

        # 获取顶点和面的索引
        vertices = np.asarray(decimated_mesh.vertices)
        triangles = np.asarray(decimated_mesh.triangles)

        # 计算每个顶点的度数
        vertex_degrees = np.zeros(len(vertices), dtype=int)
        for i in range(len(triangles)):
            v1, v2, v3 = triangles[i]
            vertex_degrees[v1] += 1
            vertex_degrees[v2] += 1
            vertex_degrees[v3] += 1

        # 找到度数为0的顶点的索引
        zero_degree_vertices = np.where(vertex_degrees == 0)[0]

        decimated_mesh.remove_vertices_by_index(zero_degree_vertices)

        _target_path =  TARGET_ROOT + "/" + _id + EXTENSION
        o3d.io.write_triangle_mesh(_target_path, decimated_mesh)

In [51]:
def drop_vertex(DATA_ROOT, TARGET_ROOT, EXTENSION):
    for file in tqdm(glob.glob(str(DATA_ROOT / "*.ply"))):
        _path = file.replace('\\', '/')

        _id = _path[_path[:_path.rfind(".")].rfind("/")+1:_path.rfind(".",0,)]

        original_mesh = o3d.io.read_triangle_mesh(_path)

        # 获取顶点和面的索引
        vertices = np.asarray(original_mesh.vertices)
        triangles = np.asarray(original_mesh.triangles)

        # 计算每个顶点的度数
        vertex_degrees = np.zeros(len(vertices), dtype=int)
        for i in range(len(triangles)):
            v1, v2, v3 = triangles[i]
            vertex_degrees[v1] += 1
            vertex_degrees[v2] += 1
            vertex_degrees[v3] += 1

        # 找到度数为0的顶点的索引
        zero_degree_vertices = np.where(vertex_degrees == 0)[0]

        original_mesh.remove_vertices_by_index(zero_degree_vertices)

        _target_path =  TARGET_ROOT + "/" + _id + EXTENSION
        o3d.io.write_triangle_mesh(_target_path, original_mesh)

In [120]:
DATA_ROOT = Path("/vol/space/projects/ukbb/projects/silhouette/registered_10")
TARGET_ROOT = "/vol/space/projects/ukbb/projects/silhouette/registered_10k_faces"
EXTENSION = ".ply"

In [121]:
drop_vertex(DATA_ROOT, TARGET_ROOT, EXTENSION)

  1%|          | 288/44200 [00:52<2:37:49,  4.64it/s]

In [54]:
DATA_ROOT = Path("/vol/space/projects/ukbb/projects/silhouette/decimated_1")
TARGET_ROOT = "/vol/space/projects/ukbb/projects/silhouette/decimated_50_faces"
FACES = 50
EXTENSION = ".ply"

In [55]:
decimate(DATA_ROOT, TARGET_ROOT, FACES, EXTENSION)

  0%|          | 0/25299 [00:00<?, ?it/s]

100%|██████████| 25299/25299 [06:59<00:00, 60.28it/s]


In [46]:
example_path = "/vol/space/projects/ukbb/projects/silhouette/registered_50_faces/1959257.ply"

In [47]:
original_mesh = o3d.io.read_triangle_mesh(example_path)

In [48]:
original_mesh.triangles

std::vector<Eigen::Vector3i> with 49 elements.
Use numpy.asarray() to access data.

In [49]:
np.array(original_mesh.vertices).shape

(27, 3)